In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


np.random.seed(0)

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
[len(df_train[df_train['label'] == i])/len(df_train) for i in range(10)]

[0.09838095238095237,
 0.11152380952380953,
 0.09945238095238096,
 0.1035952380952381,
 0.09695238095238096,
 0.09035714285714286,
 0.0985,
 0.10478571428571429,
 0.09673809523809523,
 0.09971428571428571]

Можно говорить о том, что каждый класс представлен примерно тем же количеством предсавителей, что и остальные классы.
Поэтому можно не использовать стратификацию.

In [4]:
labels = df_train['label']
df_train.drop('label', axis=1, inplace=True)

In [5]:
df_train = df_train.iloc[:,:].values
df_train = df_train.astype(np.float)

# convert from [0:255] => [0.0:1.0]
df_train = np.multiply(df_train, 1.0 / 255.0)

In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_train, labels, test_size=0.20, random_state=42)

In [7]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [8]:
y_train_one_hot = dense_to_one_hot(y_train, 10).astype(np.uint8)
y_test_one_hot = dense_to_one_hot(y_test, 10).astype(np.uint8)

## Model 1

Для начала попробуем построить простую мультиклассовую логистическую регрессию. 

Точность полученную с помощью данной модели мы будем считать за пороговое значение при сравнении с другими более сложными моделями.

### Inference

Итак, наша модель есть $$l = x \cdot W + b$$ $$ \hat{y} = \text{softmax}(l) $$

Ошибка: кросс-энтропия $$ \mathcal{L} = \sum\limits_{i=0}^{N} y_i \log \hat{y}_i $$

In [9]:
from sklearn.metrics import accuracy_score


X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name="weights")
b = tf.Variable(tf.zeros([10]), name="bias")

logits = tf.matmul(X, w) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(logits, Y)
loss = tf.reduce_mean(entropy) 

### Training

Обучаем модель методом градиентного спуска.

In [19]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

n_epochs = 100
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for i in range(n_epochs): 
    for x,y in zip(X_train.values, y_train_one_hot):
        sess.run([optimizer], feed_dict={X: [x], Y:[y]})

### Evaluation

Метрикой качества в данном случае является ```accuracy``` - процент верно предсказанных меток.

In [30]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={X: X_test.values, Y: y_test_one_hot}))
sess.close()

0.909643


## Model 2


In [10]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

На примере ниже показан сверточный уровень вместе с pooling уровнем. 

Представленные на рисунке сверточные слои (feature maps) имеют свои матрицы весов и bias'ы.
<img src="conv.png">

В нашей модели таких feature map'ов будет 32, а не 3 как на рисунке выше.

In [11]:
# Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

На первом слое используются ядра размерности [1, 5, 5], где 1 - количество каналов. Всего используется 32 различных ядра

In [12]:
W_conv1 = weight_variable([5, 5, 1, 32]) # 32 feature maps, 1 input chanel
b_conv1 = bias_variable([32]) # one per feature map

x_image = tf.reshape(X, [-1, 28, 28, 1])

# First layer (conv and max_pooling)
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Так как мы используем pooling слой после сверточного с шагом два по кадому измерению (высота/ширина), то на выходе мы имеем слой размерности [14, 14, 32], где 14,14 - высота и ширина "изображения", и 32 - количество каналов.

В рамках данного слоя каждое ядро имеет размерность [32, 5, 5], всего таких ядер на втором слое 64.

In [13]:
W_conv2 = weight_variable([5, 5, 32, 64]) # 64 feature maps, 32 input chanel
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Теперь изображение имеет размер [7, 7, 64].

Выполним линеаризацию последнего pooling слоя для того чтобы построить первый полносвязный слой, состоящий из 1024 нейронов.

In [14]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [28]:
# keep_prob = tf.placeholder(tf.float32)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Readout Layer: второй полносвязный слой, с количеством нейронов равным количеству классов. На данном уровне считается softmax логитов, для того чтобы вычислить вероятности принадлежности классам.

In [15]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

### Training

<b>Helper functions:</b>

В идеале, мы должны использовать всю обучающую выборку для процесса обучения, но это дорого с вычислительной точки зрения. Поэтому мы будем использовать батчи и сделаем процесс обучения стохастическим.

In [16]:
epochs_completed = 0
index_in_epoch = 0
num_examples = X_train.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global X_train
    global y_train_one_hot
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        X_train = X_train[perm]
        y_train_one_hot = y_train_one_hot[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return X_train[start:end], y_train_one_hot[start:end]

In [17]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

In [19]:
for i in range(2500):
    batch_xs, batch_ys = next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run([accuracy], feed_dict={X: batch_xs, Y: batch_ys})
        print('step %d, training accuracy %g' % (i, train_accuracy[0]))
    
    sess.run([train_step], feed_dict={X: batch_xs, Y: batch_ys})

step 0, training accuracy 0.12
step 100, training accuracy 0.94
step 200, training accuracy 0.84
step 300, training accuracy 0.88
step 400, training accuracy 0.98
step 500, training accuracy 0.94
step 600, training accuracy 0.94
step 700, training accuracy 0.94
step 800, training accuracy 1
step 900, training accuracy 0.96
step 1000, training accuracy 0.98
step 1100, training accuracy 1
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.98
step 2400, training accuracy 0.98


### Evaluation

In [20]:
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot}))

0.980238


In [21]:
X_validate = pd.read_csv('test.csv').values
X_validate = X_validate.astype(np.float)

# convert from [0:255] => [0.0:1.0]
X_validate = np.multiply(X_validate, 1.0 / 255.0)

In [29]:
def make_submission(preds):
    result = pd.concat([pd.DataFrame(np.array(range(1, len(X_validate)+1)), columns=['ImageId']), 
                        pd.DataFrame(preds ,columns=['Label'])], axis=1)
    result.to_csv('submission.csv', index=False)

In [24]:
prediction = tf.argmax(y_conv, 1)
# preds = sess.run([prediction], feed_dict={X: X_validate})

preds = np.zeros(X_validate.shape[0])
for i in range(X_validate.shape[0]//50):
    preds[i*50 : (i+1)*50] = sess.run([prediction], feed_dict={X: X_validate[i*50 : (i+1)*50]})[0]

In [30]:
make_submission(preds.astype(int))

In [ ]:
# sess.close()